In [1]:
# import libraries

from openai import OpenAI
import os
from dotenv import load_dotenv


In [2]:
from pathlib import Path

import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o-mini") 

##change everytime
extracted_raw_file = Path(r"work_on_minutes\acta_07_noviembre_2023\extracted_text_11_07_23_clean_vers_001.txt")

text_content = []

with open(extracted_raw_file, "r", encoding="utf-8") as text_file:
    text_full = text_file.read()
    
    # Count the number of tokens    
    text_tokens = encoding.encode(text_full)
    num_tokens = len(text_tokens)
    print(f"Number of tokens : {num_tokens}")

    text_content.append(text_full) #save for future processing
     
    print("\n ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")  
        

Number of tokens : 22264

 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [ ]:
## max input tokens 128K 
# if needed, for a long document, it is going to be necessary to break the document in pieces and make a recursive summarization 

### Prompt Engineering 1

In [ ]:
#token encoding for the instruction

# Load the encoding for the specific model you're using
encoding = tiktoken.encoding_for_model("gpt-4o-mini")    

# Instruction text
instruction = f"Has un resumen de una minuta de reunión de un gobierno local.\
La intención de este ejercicio es proveer al ciudadano común con información relevante sobre el desempeño de su gobierno local. \
Instrucciones especificas: \
- Enfatiza los problemas, peticiones y soluciones o acuerdos que se llegan \
- Indica los nombres de las personas que participan cuando sea necesario.\
- Indica datos concretos \
- No emitas juicios de valor, manten neutralidad\
- Identifica fechas\
- Identifica los temas que se trataron.\
- Verifica con cuidado al final cuantas personas asistieron a la sesion\
"

# Encode the text
instruction_tokens = encoding.encode(instruction)

# Count the number of tokens
num_tokens = len(instruction_tokens)

print(f"Number of tokens: {num_tokens}")


### Prompt Engineering 2

In [7]:
#token encoding for the instruction

# Load the encoding for the specific model you're using
encoding = tiktoken.encoding_for_model("gpt-4o-mini")    

# Instruction text
instruction = """Has un resumen de una minuta de reunión de un gobierno local con las siguientes instrucciones:
    - Identifica los principales temas que se hablaron en la sesion.
    - Enfatiza los problemas, peticiones y soluciones o acuerdos que se llegan, indicando los nombres de las personas que participan cuando sea necesario.
    - Prioriza datos concretos y no emitas juicios de valor
    - Verifica con cuidado cuantas personas asistieron a la sesion
    - Envia la informacion con la siguiente estructura:
     {
     "date" : "dd-mm-yy", 
     "document_id" : "acta_numero_del_acta", 
     "act_name": "nombre del acta 
     "topics:  ["temas tratados en la sesion"], 
      "problems"": ["problemas mencionados en la sesion"],
      "petitions": ["peticiones de los asistentes"],
      "solutions": ["soluciones o acuerdos a los que se llegaron"], 
      "participants": ["nombres de los asistentes"],
      "summary": "resumen corto de la sesion",
    } 
    """


# Encode the text
instruction_tokens = encoding.encode(instruction)

# Count the number of tokens
num_tokens = len(instruction_tokens)

print(f"Number of tokens: {num_tokens}")


Number of tokens: 233


In [4]:
# loading API key
load_dotenv()
key_ct = os.environ["ct_api_key"]

In [8]:

client = OpenAI(api_key = key_ct)

response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
        {"role": "system", "content" : instruction},
        {"role":"user", "content":text_content[0]},
    ],
    temperature = 0.70,
    max_tokens = 1100,
)


In [9]:
#change everytime
response_summary_TOTAL_vers003 = response.choices[0].message.content
print(response_summary_TOTAL_vers003)   

```json
{
  "date": "07-11-23",
  "document_id": "acta_030",
  "act_name": "Acta de la Sesión No. 030 Ordinaria del Concejo Metropolitano de Quito",
  "topics": [
    "Verificación del cuórum",
    "Presentación sobre procesos judiciales relacionados con eventos de inseguridad",
    "Informe sobre cambio de categoría de bien municipal para donación a SENESCYT",
    "Primer debate del Proyecto de Ordenanza Metropolitana del Fomento del Comercio Justo y Consumo Responsable",
    "Segundo debate de la Ordenanza para Regularización Vial en el barrio Servidores de la Salud"
  ],
  "problems": [
    "Inseguridad y colocación de artefactos explosivos en el espacio público",
    "Falta de infraestructura educativa en el sur de Quito",
    "Necesidad de regularización vial en áreas específicas"
  ],
  "petitions": [
    "Solicitudes de informes sobre procesos judiciales y reparaciones",
    "Propuestas para establecer plazos en la donación de predios a SENESCYT",
    "Requerimiento de un enfoqu

### Formatting

In [10]:
#extract ther markdown labels

import re

def extract_markdown_content(text):
    # Regular expression to find content between triple backticks
    pattern = r"```json(.*?)```"
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        return match.group(1).strip()
    else:
        return None

# Example usage

response_summary_TOTAL_vers002_formatted = extract_markdown_content(response_summary_TOTAL_vers003)
print(response_summary_TOTAL_vers002_formatted)


{
  "date": "07-11-23",
  "document_id": "acta_030",
  "act_name": "Acta de la Sesión No. 030 Ordinaria del Concejo Metropolitano de Quito",
  "topics": [
    "Verificación del cuórum",
    "Presentación sobre procesos judiciales relacionados con eventos de inseguridad",
    "Informe sobre cambio de categoría de bien municipal para donación a SENESCYT",
    "Primer debate del Proyecto de Ordenanza Metropolitana del Fomento del Comercio Justo y Consumo Responsable",
    "Segundo debate de la Ordenanza para Regularización Vial en el barrio Servidores de la Salud"
  ],
  "problems": [
    "Inseguridad y colocación de artefactos explosivos en el espacio público",
    "Falta de infraestructura educativa en el sur de Quito",
    "Necesidad de regularización vial en áreas específicas"
  ],
  "petitions": [
    "Solicitudes de informes sobre procesos judiciales y reparaciones",
    "Propuestas para establecer plazos en la donación de predios a SENESCYT",
    "Requerimiento de un enfoque partic

## Exporting to files

### Exporting to .txt 

In [11]:

#change everytime
with open(r"work_on_minutes\acta_07_noviembre_2023\response_summary_TOTAL_vers003_formatted.json", "w", encoding="utf-8") as text_file:
 
    text_file.write(response_summary_TOTAL_vers002_formatted)  # Write the block text to the file, with spacing between blocks

print("Text extraction completed and saved'.")


Text extraction completed and saved'.


### Exporting to .md

In [ ]:

#change everytime
with open(r"C:\Users\herms\Desktop\Civic_Tech_Research_Project\PreliminarWork\preprocessing\work_on_minutes\acta_26_abril_2019\summary_26_04_19\response_summary_TOTAL_vers002.md", "w", encoding="utf-8") as text_file:
 
    text_file.write(response_summary_TOTAL_vers002)  # Write the block text to the file, with spacing between blocks

print("Text extraction completed and saved to 'response_summary_TOTAL_vers002'.")


### Exporting to .json

In [ ]:

# Write the JSON data to a new file
with open(r"work_on_minutes\acta_05_diciembre_2023\response_summary_TOTAL_vers002.json", "w", encoding="utf-8") as json_file:
    json.dump( response_summary_TOTAL_vers002_formatted, json_file, ensure_ascii=False, indent=4)

print("Text extraction completed and saved as JSON.")